# Scrape Data from Shopee with beautifulsoup and selenium
### Data เพื่อการทำ Marketing ของผู้ขาย ซึ่งประกอบไปด้วย
- name, review, rating, date

!pip install selenium
!pip install bs4
!pip install pandas

# Code ที่ต้อง Import

In [15]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os  
from datetime import datetime 
import bs4
import time
import re

In [16]:
#กำหนดตัวแปร
#link ที่ต้องการดึงข้อมูล
link = 'https://www.lazada.co.th/products/90-biore-uv-aqua-rich-watery-gel-spf50-pa-90ml-i228851585.html?spm=a2o4m.searchlist.list.11.592a32030G8dMB'
#ชื่อ file output
output = 'Biore UV Aqua Rich Watery Gel'
# Set up Chrome binary_location
location = 'C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe'
#the service path to ChromeDriver
service_path = r'C:\Users\earns\Desktop\Shopee_Web_Scarping\chromedriver.exe'

In [17]:
# Set up Chrome options
options = Options()
options.binary_location = location

# Specify the path to ChromeDriver (NOT geckodriver, which is for Firefox)
service = Service(service_path)

# Initialize WebDriver with the service and options
driver = webdriver.Chrome(service=service, options=options)

# Open lazada website
driver.get(link)

#wait foe 35s.

In [18]:
#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.zoom='10%'")

In [19]:
records=[]

In [20]:
# การ Scrape ข้อมูลจากหลายหน้าของ Web Shopee
for i in range(30):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el=soup.select("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div")
    # Define star rating image sources
    src_plus_one = "//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"
    src_plus_zero = "//img.lazcdn.com/g/tps/tfs/TB18ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"

    for e in el:
        # Extract review text
        review = e.select_one("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.item-content > div.content")
        review_text = review.text.strip() if review else ""

        # Extract and calculate rating
        stars = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > div')
        rating = 0
        if stars:
            for star in stars.find_all('img'):
                if star['src'] == src_plus_one:
                    rating += 1
                elif star['src'] == src_plus_zero:
                    rating += 0

        # Extract review date
        date_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > span')
        date = date_element.text.strip() if date_element else ""

        # Extract reviewer name
        name_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.middle > span:nth-child(1)')
        name = name_element.text.strip() if name_element else ""

        # Append the extracted data to records
        records.append([name, rating, date, review_text])

    wait = WebDriverWait(driver, 20)
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]')))
    next_button.click()
    time.sleep(20)
    print(i) #print loop


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [ ]:
records

In [18]:
#เลือก filter_sort review1
wait = WebDriverWait(driver, 20)
filterStar = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[2]/div/div[2]')))
filterStar.click()

In [19]:
#เลือก sort เป็นดาวน้อยไปดาวมาก
selectStar = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div/div/ul/li[4]')))
selectStar.click()

In [12]:
# สร้าง DataFrame และเพิ่มลำดับแถว
df = pd.DataFrame(records, columns=['name', 'rating', 'date', 'review_text'])

# เพิ่มคอลัมน์ลำดับแถว (Row Index) โดยใช้ range
df['no'] = range(1, len(df) + 1)

# แสดง DataFrame ที่มีลำดับแถว
df = df[['no', 'name', 'rating', 'date', 'review_text']]


In [13]:
# รับเวลาปัจจุบัน  
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

In [14]:
# บันทึก DataFrame ลงไฟล์ CSV
csv_file = f'RawDataCSV/{output}_{current_time}.csv'
df.to_csv(csv_file, index=False)

# บันทึก DataFrame ลงไฟล์ Excel
excel_file = f'RawDataExel/{output}_{current_time}.xlsx'
df.to_excel(excel_file, index=False)